In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
# Define custom metrics and loss functions
def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.round(y_pred), tf.float32)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

# 1) GPU config & mixed precision (paste near top)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    try:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy('mixed_float16')
    except Exception:
        pass

# Load the trained segmentation model
model = tf.keras.models.load_model(
    'Trained_Model - (OriginalPARAM)/road_segmentation_unet700img.keras',
    custom_objects={
        'iou_metric': iou_metric
    }
)

# after loading your model
@tf.function
def model_infer(x):
    # ensure input dtype matches model expectation (float32 if not using mixed)
    return model(x, training=False)

# Path to the dashcam video
video_path = "roaddashcamfootage_day.MP4"
cap = cv2.VideoCapture(video_path)
save_output = False
output_path = "segmented_output.mp4"

if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Smoothing
prev_mask = None
alpha = 0.6  # temporal smoothing strength
kernel = np.ones((5, 5), np.uint8)  # morphological smoothing kernel

# Main loop to process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # --- ROI: Focus only on lower half (road area) ---
    h, w, _ = frame.shape
    roi = frame[int(h*0.5):h, 0:w]   # bottom 50% of the image


    # Prepare input for model
    resized_frame = cv2.resize(roi, (256, 256))
    inp = (resized_frame.astype(np.float32) / 255.0)[None, ...]
    inp_tf = tf.convert_to_tensor(inp)

    # Predict mask (GPU)
    pred_mask = model_infer(inp_tf).numpy()[0]

    # Convert to binary
    binary_mask = (pred_mask[:, :, 0] > 0.5).astype(np.uint8)

    # ---- Morphological smoothing ----
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)

    # Resize mask back to original frame size
    mask_resized = cv2.resize(binary_mask, (w, h//2), interpolation=cv2.INTER_NEAREST)

        # ---- Temporal smoothing ----
    if prev_mask is None:
        smooth_mask = mask_resized.astype(np.float32)
    else:
        smooth_mask = alpha * prev_mask + (1 - alpha) * mask_resized

    prev_mask = smooth_mask
    mask_resized = (smooth_mask > 0.5).astype(np.uint8)

    #Place ROI mask into full-frame mask
    full_mask = np.zeros((h, w), dtype=np.uint8)
    full_mask[h//2:h, 0:w] = mask_resized

    #Overlay mask
    overlay = frame.copy()
    overlay[full_mask == 1] = [0, 255, 0]
    output_frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)

    # Create resizable window
    cv2.namedWindow('Road Segmentation', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Road Segmentation', 1600, 900)

    # Display the output
    cv2.imshow('Road Segmentation', output_frame)

    # Save output video frame (if enabled)
    if save_output:
        out.write(output_frame)

    # Press 'q' to exit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Cleanup
cap.release()
if save_output:
    out.release()
cv2.destroyAllWindows()